In [11]:
import os
import random
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
# Specify the path to the directory containing the images
image_dir = "../datasets/ndvi_images"

# Get a list of all the image files in the directory
image_files = [os.path.join(image_dir, file)
               for file in os.listdir(image_dir) if file.endswith(".tiff")]

# Select a random image file from the list
random_image = random.choice(image_files)

In [ ]:
plt.imshow(np.array(Image.open(random_image)), cmap='gray')
plt.axis('off')
plt.colorbar()
plt.show()

In [ ]:
# show the 3 separate channels of the image in 3 separate plots
image = np.array(Image.open(random_image))
fig, ax = plt.subplots(1, 4, figsize=(15, 5))
for i in range(4):
    ax[i].imshow(image[:, :, i], cmap='gray')
    ax[i].axis('off')
    ax[i].set_title(f'Channel {i}')